In [1]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 42.6 MB/s eta 0:00:00


In [19]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json

In [3]:
with open('/content/statistical_number_of_people.json', 'r') as f:
  peopls_number = json.load(f)
peopls_number

{'kindergartens': 661701, 'Pupils': 1725220, 'adults': 10268129}

In [4]:
df_buildings = gpd.read_file('/content/buildings_transform.geojson')
df_buildings

,id,addr:city,addr:housenumber,addr:postcode,addr:street,building,building:flats,building:levels,design:ref,centroid latitude,centroid longitude,area,district_id,district_name,year,region_id,region_name,geometry
0,relation/58174,Москва,6,000000,улица Довженко,apartments,912,10,1-515/9М,55.722578,37.513598,"46 231,00",relation/446115,район Раменки,1976,relation/226149,Западный административный округ,"POLYGON ((37.51454 55.72195, 37.51475 55.72208..."
1,relation/1822659,Москва,1 кБ,000000,empty,dormitory,0,18,empty,55.702349,37.531776,3557.367502257228,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53149 55.70189, 37.53160 55.70195..."
2,relation/1822660,Москва,1 кВ,000000,empty,dormitory,0,18,empty,55.703472,37.529818,3498.702996812761,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53004 55.70388, 37.53017 55.70381..."
3,relation/1822661,Москва,1 кГ,000000,empty,dormitory,0,9,empty,55.701592,37.531839,1621.687709802762,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53160 55.70195, 37.53149 55.70189..."
4,relation/1822662,Москва,1 кД,000000,empty,dormitory,0,9,empty,55.703482,37.528478,1624.636643284932,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.52896 55.70328, 37.52903 55.70332..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33999,way/816879973,Москва,4 к2,111677,улица Вертолётчиков,apartments,256,17,empty,55.704216,37.944299,1308.855795994401,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94449 55.70461, 37.94378 55.70379..."
34000,way/821336533,Москва,12 к5,000000,улица Маресьева,apartments,0,14,empty,55.713050,37.943830,1274.007280081511,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94334 55.71300, 37.94407 55.71280..."
34001,way/844852891,Москва,6 к1,111677,улица Маресьева,apartments,231,22,empty,55.709866,37.936203,770.2242055386305,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.93633 55.71006, 37.93579 55.70972..."
34002,way/916147076,Москва,10 к2,000000,улица Маресьева,apartments,0,17,empty,55.712484,37.940740,1089.270644754171,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94094 55.71267, 37.94078 55.71276..."


In [5]:
median_level = 9.0
basic_footprint = 1000

In [6]:
list_of_levels = list(df_buildings['building:levels'])
print(len(list_of_levels))
list_of_levels.count('')
for i in range(len(list_of_levels)):
  #Есть 6 багов в этажности, инфы никакой, тупо игнорим и ставим среднее значение этажности
  try:
    list_of_levels[i] = float(list_of_levels[i])
    if list_of_levels[i] == 0:
      list_of_levels[i] = median_level
  except ValueError:
    #print(list_of_levels[i])
    list_of_levels[i] = median_level
print(len(list_of_levels))
list_of_levels.count(0)

34004
34004


0

In [7]:
list_of_areas = list(df_buildings['area'])
for i in range(len(list_of_areas)):
  if '.' in list_of_areas[i] and ' ' not in list_of_areas[i]:
    try:
      list_of_areas[i] = float(list_of_areas[i]) *  list_of_levels[i]
    except ValueError:
      #print(1, list_of_areas[i])
      list_of_areas[i] = basic_footprint * median_level
  else:
    try:
      list_of_areas[i] = list_of_areas[i].replace(' ', '')
      list_of_areas[i] = float(list_of_areas[i].replace(',', '.'))
    except ValueError:
      #print(2, list_of_areas[i])
      list_of_areas[i] = basic_footprint * median_level
len(list_of_areas)

34004

In [8]:
df_buildings['area'] = list_of_areas
df_buildings

,id,addr:city,addr:housenumber,addr:postcode,addr:street,building,building:flats,building:levels,design:ref,centroid latitude,centroid longitude,area,district_id,district_name,year,region_id,region_name,geometry
0,relation/58174,Москва,6,000000,улица Довженко,apartments,912,10,1-515/9М,55.722578,37.513598,46231.000000,relation/446115,район Раменки,1976,relation/226149,Западный административный округ,"POLYGON ((37.51454 55.72195, 37.51475 55.72208..."
1,relation/1822659,Москва,1 кБ,000000,empty,dormitory,0,18,empty,55.702349,37.531776,64032.615041,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53149 55.70189, 37.53160 55.70195..."
2,relation/1822660,Москва,1 кВ,000000,empty,dormitory,0,18,empty,55.703472,37.529818,62976.653943,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53004 55.70388, 37.53017 55.70381..."
3,relation/1822661,Москва,1 кГ,000000,empty,dormitory,0,9,empty,55.701592,37.531839,14595.189388,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53160 55.70195, 37.53149 55.70189..."
4,relation/1822662,Москва,1 кД,000000,empty,dormitory,0,9,empty,55.703482,37.528478,14621.729790,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.52896 55.70328, 37.52903 55.70332..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33999,way/816879973,Москва,4 к2,111677,улица Вертолётчиков,apartments,256,17,empty,55.704216,37.944299,22250.548532,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94449 55.70461, 37.94378 55.70379..."
34000,way/821336533,Москва,12 к5,000000,улица Маресьева,apartments,0,14,empty,55.713050,37.943830,17836.101921,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94334 55.71300, 37.94407 55.71280..."
34001,way/844852891,Москва,6 к1,111677,улица Маресьева,apartments,231,22,empty,55.709866,37.936203,16944.932522,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.93633 55.71006, 37.93579 55.70972..."
34002,way/916147076,Москва,10 к2,000000,улица Маресьева,apartments,0,17,empty,55.712484,37.940740,18517.600961,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94094 55.71267, 37.94078 55.71276..."


In [9]:
total_area_sum = df_buildings['area'].sum()
total_area_sum

324555391.044603

In [10]:
density_of_kindergartens = peopls_number['kindergartens'] / total_area_sum
density_of_pupils = peopls_number['Pupils'] / total_area_sum
density_of_adults = peopls_number['adults'] / total_area_sum

In [11]:
total_density = (peopls_number['kindergartens'] + peopls_number['Pupils'] + peopls_number['adults']) / total_area_sum
total_density

0.03899195745684237

In [12]:
print(density_of_kindergartens, density_of_pupils, density_of_adults)

0.0020387922008328735 0.00531564117436862 0.03163752408164088


In [13]:
df_buildings['kindergartens'] = ''
df_buildings['Pupils'] = ''
df_buildings['adults'] = ''

In [14]:
for i in range(df_buildings.shape[0]):
  df_buildings.loc[i, 'kindergartens'] = round(df_buildings.loc[i]['area'] * density_of_kindergartens)
  df_buildings.loc[i, 'Pupils'] = round(df_buildings.loc[i]['area'] * density_of_pupils)
  df_buildings.loc[i, 'adults'] = round(df_buildings.loc[i]['area'] * density_of_adults)

In [15]:
df_buildings

,id,addr:city,addr:housenumber,addr:postcode,addr:street,building,building:flats,building:levels,design:ref,centroid latitude,...,area,district_id,district_name,year,region_id,region_name,geometry,kindergartens,Pupils,adults
0,relation/58174,Москва,6,000000,улица Довженко,apartments,912,10,1-515/9М,55.722578,...,46231.000000,relation/446115,район Раменки,1976,relation/226149,Западный административный округ,"POLYGON ((37.51454 55.72195, 37.51475 55.72208...",94,246,1463
1,relation/1822659,Москва,1 кБ,000000,empty,dormitory,0,18,empty,55.702349,...,64032.615041,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53149 55.70189, 37.53160 55.70195...",131,340,2026
2,relation/1822660,Москва,1 кВ,000000,empty,dormitory,0,18,empty,55.703472,...,62976.653943,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53004 55.70388, 37.53017 55.70381...",128,335,1992
3,relation/1822661,Москва,1 кГ,000000,empty,dormitory,0,9,empty,55.701592,...,14595.189388,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53160 55.70195, 37.53149 55.70189...",30,78,462
4,relation/1822662,Москва,1 кД,000000,empty,dormitory,0,9,empty,55.703482,...,14621.729790,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.52896 55.70328, 37.52903 55.70332...",30,78,463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33999,way/816879973,Москва,4 к2,111677,улица Вертолётчиков,apartments,256,17,empty,55.704216,...,22250.548532,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94449 55.70461, 37.94378 55.70379...",45,118,704
34000,way/821336533,Москва,12 к5,000000,улица Маресьева,apartments,0,14,empty,55.713050,...,17836.101921,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94334 55.71300, 37.94407 55.71280...",36,95,564
34001,way/844852891,Москва,6 к1,111677,улица Маресьева,apartments,231,22,empty,55.709866,...,16944.932522,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.93633 55.71006, 37.93579 55.70972...",35,90,536
34002,way/916147076,Москва,10 к2,000000,улица Маресьева,apartments,0,17,empty,55.712484,...,18517.600961,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94094 55.71267, 37.94078 55.71276...",38,98,586


In [16]:
with open('/content/Отклонения.txt', 'r') as f:
  string_of_number = f.read()
  string_of_number.strip()
string_of_number

'88 67 89 68 96 109 102 67 114 134 98 97 70 140 83 63 139 65 79 76 105 120 157 75 65 75 101 114 76 72 161 72 97 83 119 148 96 103 136 85 76 80 84 100 99 91 76 86 108 96 101 105 86 96 82 81 101 72 101 70'

In [ ]:
list_of_number = string_of_number.split(' ')
list_of_number = [int(i) for i in list_of_number]
list_of_number

In [22]:
len(list_of_number)

60

In [ ]:
m = 0
add_list = []
for i in list_of_number:
  add_list.append(100 - i)
add_list

In [27]:
mean = np.mean(add_list)
mean

5.083333333333333

In [20]:
std = np.std(list_of_number)
std

23.38039896057284

In [ ]:
#Получаем что в среднем мы ошибаем на 5 процентов. Пересчитываем

In [31]:
correction_coef = 1.05083

In [32]:
density_of_kindergartens = (peopls_number['kindergartens'] / total_area_sum) * correction_coef
density_of_pupils = (peopls_number['Pupils'] / total_area_sum) * correction_coef
density_of_adults = (peopls_number['adults'] / total_area_sum) * correction_coef

In [33]:
total_density = ((peopls_number['kindergartens'] + peopls_number['Pupils'] + peopls_number['adults']) / total_area_sum) * correction_coef
total_density

0.040973918654373664

In [34]:
print(density_of_kindergartens, density_of_pupils, density_of_adults)

0.0021424240084012085 0.005585835215261776 0.033245659430710686


In [35]:
df_buildings['kindergartens'] = ''
df_buildings['Pupils'] = ''
df_buildings['adults'] = ''

In [36]:
for i in range(df_buildings.shape[0]):
  df_buildings.loc[i, 'kindergartens'] = round(df_buildings.loc[i]['area'] * density_of_kindergartens)
  df_buildings.loc[i, 'Pupils'] = round(df_buildings.loc[i]['area'] * density_of_pupils)
  df_buildings.loc[i, 'adults'] = round(df_buildings.loc[i]['area'] * density_of_adults)

In [37]:
df_buildings

,id,addr:city,addr:housenumber,addr:postcode,addr:street,building,building:flats,building:levels,design:ref,centroid latitude,...,area,district_id,district_name,year,region_id,region_name,geometry,kindergartens,Pupils,adults
0,relation/58174,Москва,6,000000,улица Довженко,apartments,912,10,1-515/9М,55.722578,...,46231.000000,relation/446115,район Раменки,1976,relation/226149,Западный административный округ,"POLYGON ((37.51454 55.72195, 37.51475 55.72208...",99,258,1537
1,relation/1822659,Москва,1 кБ,000000,empty,dormitory,0,18,empty,55.702349,...,64032.615041,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53149 55.70189, 37.53160 55.70195...",137,358,2129
2,relation/1822660,Москва,1 кВ,000000,empty,dormitory,0,18,empty,55.703472,...,62976.653943,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53004 55.70388, 37.53017 55.70381...",135,352,2094
3,relation/1822661,Москва,1 кГ,000000,empty,dormitory,0,9,empty,55.701592,...,14595.189388,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.53160 55.70195, 37.53149 55.70189...",31,82,485
4,relation/1822662,Москва,1 кД,000000,empty,dormitory,0,9,empty,55.703482,...,14621.729790,relation/446115,район Раменки,NaN,relation/226149,Западный административный округ,"POLYGON ((37.52896 55.70328, 37.52903 55.70332...",31,82,486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33999,way/816879973,Москва,4 к2,111677,улица Вертолётчиков,apartments,256,17,empty,55.704216,...,22250.548532,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94449 55.70461, 37.94378 55.70379...",48,124,740
34000,way/821336533,Москва,12 к5,000000,улица Маресьева,apartments,0,14,empty,55.713050,...,17836.101921,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94334 55.71300, 37.94407 55.71280...",38,100,593
34001,way/844852891,Москва,6 к1,111677,улица Маресьева,apartments,231,22,empty,55.709866,...,16944.932522,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.93633 55.71006, 37.93579 55.70972...",36,95,563
34002,way/916147076,Москва,10 к2,000000,улица Маресьева,apartments,0,17,empty,55.712484,...,18517.600961,relation/1255577,район Некрасовка,NaN,relation/1278703,Юго-Восточный административный округ,"POLYGON ((37.94094 55.71267, 37.94078 55.71276...",40,103,616


In [38]:
df_buildings.to_file('number_of_people_buildings.geojson', driver="GeoJSON")